# spark parquet merge schema and partition pruning

In [3]:
df = spark.read.load("/home/somanath/spark/spark-2.2.0-bin-hadoop2.7/examples/src/main/resources/users.parquet")

In [28]:
spark_home="/home/somanath/spark/spark-2.2.0-bin-hadoop2.7/"

In [5]:
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [6]:
df.write.partitionBy("favorite_color").format("parquet").save("namesPartByColor.parquet")

In [8]:
!ls namesPartByColor.parquet

favorite_color=__HIVE_DEFAULT_PARTITION__  favorite_color=red  _SUCCESS


In [9]:
df2=spark.read.load("namesPartByColor.parquet")

In [14]:
import pyspark.sql.functions as f

In [17]:
#only one partition is read
df2.select("*").filter(f.col("favorite_color") == "red").explain()

== Physical Plan ==
*FileScan parquet [name#30,favorite_numbers#31,favorite_color#32] Batched: false, Format: Parquet, Location: InMemoryFileIndex[file:/home/somanath/namesPartByColor.parquet], PartitionCount: 1, PartitionFilters: [isnotnull(favorite_color#32), (favorite_color#32 = red)], PushedFilters: [], ReadSchema: struct<name:string,favorite_numbers:array<int>>


In [18]:
#all partition is read
df2.select("*").explain()

== Physical Plan ==
*FileScan parquet [name#30,favorite_numbers#31,favorite_color#32] Batched: false, Format: Parquet, Location: InMemoryFileIndex[file:/home/somanath/namesPartByColor.parquet], PartitionCount: 2, PartitionFilters: [], PushedFilters: [], ReadSchema: struct<name:string,favorite_numbers:array<int>>


In [30]:
df = spark.sql("SELECT * FROM parquet.`{}examples/src/main/resources/users.parquet`".format(spark_home))

In [31]:
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [41]:
!mkdir -p /home/somanath/data

mkdir: cannot create directory ‘/home/somanath/data’: File exists


In [33]:
from pyspark.sql import Row

In [42]:
squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                      .map(lambda i: Row(single=i, double=i ** 2)))
squaresDF.show()
squaresDF.write.parquet("/home/somanath/parquetmerge/test_table/key=1")

    #

    

+------+------+
|double|single|
+------+------+
|     1|     1|
|     4|     2|
|     9|     3|
|    16|     4|
|    25|     5|
+------+------+



In [43]:
# Create another DataFrame in a new partition directory,
    # adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                    .map(lambda i: Row(single=i, triple=i ** 3)))
cubesDF.show()
cubesDF.write.parquet("/home/somanath/parquetmerge/test_table/key=2")


+------+------+
|single|triple|
+------+------+
|     6|   216|
|     7|   343|
|     8|   512|
|     9|   729|
|    10|  1000|
+------+------+



In [47]:
spark.sql("set  spark.sql.parquet.mergeSchema ").show()

+--------------------+-----+
|                 key|value|
+--------------------+-----+
|spark.sql.parquet...|false|
+--------------------+-----+



In [45]:
spark.read.parquet("/home/somanath/parquetmerge/test_table").show()

+------+------+---+
|double|single|key|
+------+------+---+
|  null|     9|  2|
|  null|    10|  2|
|    16|     4|  1|
|    25|     5|  1|
|  null|     8|  2|
|  null|     6|  2|
|  null|     7|  2|
|     4|     2|  1|
|     1|     1|  1|
|     9|     3|  1|
+------+------+---+



In [52]:
# Read the partitioned table
mergedDF = spark.read.option("mergeSchema", "true").parquet("/home/somanath/parquetmerge/test_table")
mergedDF.printSchema()
mergedDF.show()

root
 |-- double: long (nullable = true)
 |-- single: long (nullable = true)
 |-- triple: long (nullable = true)
 |-- key: integer (nullable = true)

+------+------+------+---+
|double|single|triple|key|
+------+------+------+---+
|  null|     9|   729|  2|
|  null|    10|  1000|  2|
|    16|     4|  null|  1|
|    25|     5|  null|  1|
|  null|     8|   512|  2|
|  null|     6|   216|  2|
|  null|     7|   343|  2|
|     4|     2|  null|  1|
|     1|     1|  null|  1|
|     9|     3|  null|  1|
+------+------+------+---+



In [53]:
#inference we can see the merge schema reading both tables